In [2]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00


In [3]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00


In [11]:
import torch
from transformers import pipeline
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download("stopwords")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [54]:
dataset = load_dataset('cnn_dailymail', '3.0.0')

In [19]:
print(dataset["test"][0]["article"])

(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, sa

# TF-IDF Vectorizer

In [96]:
document = dataset["test"][0]["article"].strip()

# Tokenize the document into sentences
sentences = nltk.sent_tokenize(document)

# Extract stopwords and preprocess sentences
stop_words = set(stopwords.words("english"))
sentences = [nltk.word_tokenize(sentence.lower()) for sentence in sentences]
sentences = [' '.join([token for token in tokens if token.isalnum() and token not in stop_words]) for tokens in sentences]

# Calculate TF-IDF features
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sentences)

# Calculate cosine similarity between sentences
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Get sentence scores
sentence_scores = cosine_matrix.sum(axis=1)

# Select the top N sentences as the summary
num_sentences_in_summary = 3
selected_sentences = [sentences[i] for i in sentence_scores.argsort()[-num_sentences_in_summary:][::-1]]

# Print the extractive summary
print("\nExtractive Summary:")
for sentence in selected_sentences:
    print(sentence)


Extractive Summary:
cnn palestinian authority officially became 123rd member international criminal court wednesday step gives court jurisdiction alleged crimes palestinian territories
palestine formally becomes state party rome statute today world also step closer ending long era impunity injustice said according icc news release
palestinians signed icc founding rome statute january also accepted jurisdiction alleged crimes committed occupied palestinian territory including east jerusalem since june 13 2014


# Summarization using BART

In [68]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [74]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [81]:
input_tensor = tokenizer.encode(document, return_tensors="pt", max_length=512)

outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
outputs_tensor

tensor([[    2,     0,   133,  5791,  4305,  3374,     5, 17072,  2586,   919,
             9,     5,  1016, 10203,   837,     4,    20,   517,  2029,     5,
           461, 10542,    81,  1697,  3474,    11,  5791, 13560,     4,  1870,
             8,     5,   315,   532,     6,  5063,     9,    61,    16,    41,
         14305,   919,     6,  4340,     5,  8345,   108,  1170,     7,  1962,
             5,   809,     4,    20,  4828,   899,  1499,    21,  4760,    19,
            10,  2844,    23,    20, 24852,     6,    11,     5,  7556,     6,
           147,     5,   461,    16,   716,     4,   113,  5625,  3291,   201,
          2789,     7,    84,  1373,  1175,     9,  2427,     8,  1987,    60,
          5791,  3125,   692, 16221,   625,  1076,    12,   448,  9707,   118,
            26,    23,     5,  2844,     4,    22,  1620,    52,    33,    26,
          3987,     6,    52,   109,    45,   679,    14, 16398,    16,    10,
           194,    60,     5,   121,     4,   104,  

The outputs will be a tensor in order to get text out of it, we need to decode it using the same Bart Tokenizer model.

In [82]:
print(tokenizer.decode(outputs_tensor[0]))

</s><s>The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based."Today brings us closer to our shared goals of justice and peace," Palestinian Foreign Minister Riad al-Malki said at the ceremony. "As we have said repeatedly, we do not believe that Palestine is a state," the U.S. State Department said.</s>


# Summarization using transformers’ summarization pipeline

This is a low-level or basic implementation of the summarization model using transformers.

In [79]:
from transformers import pipeline

summarizer = pipeline('summarization')

summary = summarizer(document, max_length = 180, min_length =  30)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [80]:
summary

[{'summary_text': " The Palestinian Authority becomes the 123rd member of the International Criminal Court . The move gives the court jurisdiction over alleged crimes in Palestinian territories . Israel and the United States opposed the Palestinians' efforts to join the court ."}]

Summarization using BART, a pre-trained transformer-based model, often yields better results compared to TF-IDF vectorizer summarization. BART is designed to understand context and generate coherent summaries, making it more effective at capturing the essence of a text.

Additionally, fine-tuning BART on specific summarization tasks or domains can further improve the quality of summaries. Fine-tuning allows the model to adapt to the specific requirements and nuances of the summarization task, resulting in more accurate and contextually relevant summaries.

In summary, BART-based summarization generally outperforms TF-IDF vectorization, and fine-tuning can enhance its performance even further, making it a preferred choice for various summarization tasks.